In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import time
from scipy import stats
import re

# Read in data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
# products = pd.read_csv('/content/drive/MyDrive/SocialNetworkAnalysis-2024Spring/Final Project/data/final_products_uk.csv') # Jiaying path
products = pd.read_csv('/content/drive/MyDrive/Social Network Analysis/data/final_products_uk_trimmed.csv') # Sovann path

In [17]:
products.tail()

,id,price,full_desc
121924,B074M8K16V,8.99,title:ILOVEFANCYDRESS ADULT LADIES BLACK POLIC...
121925,B0BF5555Q3,8.99,title:7.8inch Silicone Air Fryer Liners Reusab...
121926,B084R1YXKQ,1.60,title:Lindt EXCELLENCE Dark 90% Intense Cacao ...
121927,B09VCZ4G6S,29.99,title:Yaheetech Metal Bird Cage with Open Play...
121928,1529393833,4.50,"title:Before I Do: the new, funny and unexpect..."


# Desc encoding

In [27]:
!pip install -q sentence_transformers
from sentence_transformers import SentenceTransformer


In [28]:
model = SentenceTransformer("all-MiniLM-L6-v2")

# sentences we like to encode
sentences = list(products['full_desc'])

# Sentences are encoded by calling model.encode()
embeddings = model.encode(sentences)

In [30]:
embeddings.shape

(121929, 384)

In [33]:
embeddings_df = pd.DataFrame(embeddings)
embeddings_df

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
0,-0.133357,0.074655,0.013319,-0.034830,-0.022844,-0.018001,0.062056,0.037391,-0.033554,-0.019909,...,0.006352,-0.030656,-0.019208,-0.061660,0.000790,0.038441,0.082899,-0.111414,0.007166,0.093550
1,-0.070727,0.046415,0.043782,0.014466,-0.051082,-0.030673,0.088054,0.040306,-0.063348,-0.008518,...,0.011188,0.013657,-0.012571,0.017447,0.035623,0.040479,0.002332,0.057068,-0.044040,0.025917
2,-0.024233,-0.060561,-0.069410,-0.026652,0.032268,0.045401,0.028998,0.004713,-0.007779,0.042547,...,0.049161,-0.049582,-0.019285,-0.019245,-0.008358,0.003566,0.069167,-0.093249,-0.045561,-0.021786
3,-0.018058,0.018740,0.048496,0.001538,0.041883,0.009186,0.054699,-0.025791,-0.034513,0.001821,...,0.009783,-0.071129,-0.067153,-0.053965,0.048870,0.050820,0.033673,-0.028969,-0.017347,0.062688
4,-0.025332,0.017676,0.048878,0.006857,0.006229,-0.015775,0.056113,-0.065923,-0.061704,-0.019692,...,-0.006476,0.053061,0.000386,0.037455,-0.014290,-0.000013,0.106422,-0.047063,0.016594,0.025755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121924,-0.051917,0.064089,0.014681,0.039744,0.054105,0.008675,0.085101,-0.043275,-0.004136,0.019331,...,-0.054479,-0.021931,0.025235,0.039864,-0.089239,0.036310,0.027649,-0.054071,-0.018608,-0.057744
121925,-0.136261,0.046591,0.040517,0.069551,0.040360,-0.006348,0.069317,0.028165,0.072422,-0.078415,...,-0.058460,-0.022162,-0.037689,0.021972,0.089832,0.018118,0.048723,-0.010600,-0.008979,0.053898
121926,-0.000234,-0.058319,0.021647,0.035110,0.015804,0.030854,0.002108,0.030791,-0.002996,-0.008640,...,-0.089446,-0.002287,-0.041225,-0.012945,-0.027538,-0.026004,0.070075,-0.145664,0.009884,0.036154
121927,0.057621,0.032720,-0.028070,-0.037975,-0.051438,-0.005115,0.136044,-0.077735,0.097110,0.036560,...,-0.062379,0.085338,0.046300,-0.007732,0.040628,-0.059024,-0.064590,-0.008735,-0.021030,0.050464


In [42]:
products_embeddings = pd.concat([products, embeddings_df], axis=1).drop(columns="full_desc")
products_embeddings

,id,price,0,1,2,3,4,5,6,7,...,374,375,376,377,378,379,380,381,382,383
0,B07SZHKGZJ,11.99,-0.133357,0.074655,0.013319,-0.034830,-0.022844,-0.018001,0.062056,0.037391,...,0.006352,-0.030656,-0.019208,-0.061660,0.000790,0.038441,0.082899,-0.111414,0.007166,0.093550
1,B0B1DW1GH1,12.99,-0.070727,0.046415,0.043782,0.014466,-0.051082,-0.030673,0.088054,0.040306,...,0.011188,0.013657,-0.012571,0.017447,0.035623,0.040479,0.002332,0.057068,-0.044040,0.025917
2,B089X4XCLB,36.85,-0.024233,-0.060561,-0.069410,-0.026652,0.032268,0.045401,0.028998,0.004713,...,0.049161,-0.049582,-0.019285,-0.019245,-0.008358,0.003566,0.069167,-0.093249,-0.045561,-0.021786
3,B08P1J79P6,24.99,-0.018058,0.018740,0.048496,0.001538,0.041883,0.009186,0.054699,-0.025791,...,0.009783,-0.071129,-0.067153,-0.053965,0.048870,0.050820,0.033673,-0.028969,-0.017347,0.062688
4,B07PNBPVN1,3.75,-0.025332,0.017676,0.048878,0.006857,0.006229,-0.015775,0.056113,-0.065923,...,-0.006476,0.053061,0.000386,0.037455,-0.014290,-0.000013,0.106422,-0.047063,0.016594,0.025755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121924,B074M8K16V,8.99,-0.051917,0.064089,0.014681,0.039744,0.054105,0.008675,0.085101,-0.043275,...,-0.054479,-0.021931,0.025235,0.039864,-0.089239,0.036310,0.027649,-0.054071,-0.018608,-0.057744
121925,B0BF5555Q3,8.99,-0.136261,0.046591,0.040517,0.069551,0.040360,-0.006348,0.069317,0.028165,...,-0.058460,-0.022162,-0.037689,0.021972,0.089832,0.018118,0.048723,-0.010600,-0.008979,0.053898
121926,B084R1YXKQ,1.60,-0.000234,-0.058319,0.021647,0.035110,0.015804,0.030854,0.002108,0.030791,...,-0.089446,-0.002287,-0.041225,-0.012945,-0.027538,-0.026004,0.070075,-0.145664,0.009884,0.036154
121927,B09VCZ4G6S,29.99,0.057621,0.032720,-0.028070,-0.037975,-0.051438,-0.005115,0.136044,-0.077735,...,-0.062379,0.085338,0.046300,-0.007732,0.040628,-0.059024,-0.064590,-0.008735,-0.021030,0.050464


In [43]:
products_embeddings.to_csv('/content/drive/MyDrive/Social Network Analysis/data/products_w_embeddings.csv') # Sovann path